In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
%%time
train_metadata_kaggle = dd.read_csv('mydata_train_metadata.csv')
test_metadata_kaggle = dd.read_csv('mydata_test_metadata.csv')
train_metadata_kaggle = train_metadata_kaggle.compute()
test_metadata_kaggle = test_metadata_kaggle.compute()
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 142) (3492890, 141)
CPU times: user 3min 30s, sys: 12.9 s, total: 3min 43s
Wall time: 41.7 s


In [5]:
%%time
train_metadata = dd.read_csv('train_metadata_final.csv')
test_metadata = dd.read_csv('test_metadata_final.csv')
train_metadata = train_metadata.compute()
test_metadata = test_metadata.compute()
print(train_metadata.shape,test_metadata.shape)

(7848, 233) (3492890, 232)
CPU times: user 3min 49s, sys: 12.9 s, total: 4min 2s
Wall time: 45 s


In [6]:
%%time
test_metadata_kaggle = test_metadata_kaggle.reset_index(drop=True)
test_metadata = test_metadata.reset_index(drop=True)

CPU times: user 6.55 s, sys: 9.59 s, total: 16.1 s
Wall time: 16.1 s


In [7]:
print(train_metadata_kaggle['object_id'].equals(train_metadata['object_id']))

True


In [8]:
print(test_metadata_kaggle['object_id'].equals(test_metadata['object_id']))

True


In [9]:
train_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_pl

In [10]:
train_metadata_kaggle.head()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,haversine,latlon1,hostgal_photoz_certain,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___2_,__median_absolute_deviation___2_,__qso_log_chi2_qsonu___0_,__stetson_k___1_,__freq1_signif___2_,__stetson_k___2_,__freq3_amplitude1___1_,__median_absolute_deviation___2_.1,__percent_close_to_median___2_,__freq_varrat___5_,__freq_varrat___4_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___5_,__std___4_,__freq_varrat___3_,__amplitude___2_,outlierScore,hipd,lipd,highEnergy_transitory_1.0_TF,highEnergy_transitory_1.5_TF,lowEnergy_transitory_1.0_TF,lowEnergy_transitory_1.5_TF,A1_minus_3_sigma,A5_max_median_diff_flux,A5_minus_3_sigma,A5_max_mean_diff_flux,diff_A5_A4_max_min_flux,diff_A2_A1_max_min_flux,diff_A3_A2_median_min_flux,diff_A5_A4_max_median_flux,diff_A4_A3_max_median_flux,diff_A2_A0_median_min_flux,diff_A4_A3_max_mean_flux,diff_A5_A2_max_mean_flux,diff_A5_A3_max_mean_flux,diff_A4_A0_median_mean_flux,diff_A5_A4_max_mean_flux,diff_A2_A1_max_median_flux,diff_A5_A2_max_median_flux,diff_A5_A4_median_min_flux,diff_A4_A0_median_min_flux,diff_A4_A1_max_median_flux,diff_A4_A2_max_median_flux,diff_A5_A4_minus_1_sigma,diff_A5_A3_median_min_flux,diff_A5_A3_max_median_flux,diff_A3_A1_minus_1_sigma,diff_A3_A0_median_min_flux,diff_A3_A0_plus_1_sigma,diff_A1_A0_median_min_flux,diff_A4_A2_mean_min_flux,diff_A5_A1_plus_1_sigma,diff_A4_A1_median_mean_flux,diff_A3_A2_max_median_flux,diff_A5_A1_median_mean_flux,div_A4_A2_median_min_flux,div_A5_A2_median_min_flux,div_A5_A2_minus_1_sigma,div_A5_A4_median_mean_flux,div_A3_A0_plus_1_sigma,div_A4_A1_minus_1_sigma,A5_median_mean_avg_flux,A4_mean_min_avg_flux,A0_max_mean_avg_flux
0,615,-1100.440063,660.626343,-123.096998,-89.477524,394.109851,-0.349540,2.130510,12.845472,4.482743,3.835268,1.744747,1.623740,0.946023,2.929669e+06,0.812722,-9.601766e+08,-1.414322,-327.742307,1761.066406,-14.306331,-5.373326,205.036926,1628.427737,-1.475181,0.128917,22370.594834,2806.374162,-1.255123,0.415580,7780.500807,2805.598113,-1.409885,0.339918,7024.003068,2536.068846,-1.449858,0.293128,3245.366349,2741.539785,-1.548319,0.200096,2704.641265,2893.344217,-1.592820,0.125268,352.0,19.0,29.0,202.114067,1.999688,35.0,4.0,2.631898,2.631898,873.7903,0.0000,0.0000,NaN,0.017,92,0.319006,-1.528827,0.000000,-205.03693,-3.254554,0.3528,0.9653,57,0.9830,0.2854,0.9120,0.362,-682.000,52,5.562230e-26,6.719410e-20,368.12900,6.217890,1.091730,5.49891,1.053490,114.465000,368.12900,0.172414,0.401664,0.129578,9.16612,9.508750,7.344980,289.27700,0.110785,64

In [11]:
%%time
for i in list(train_metadata)[131:179]:
    train_metadata[i].fillna(0.0,inplace=True)
    test_metadata[i].fillna(0.0,inplace=True)

CPU times: user 560 ms, sys: 0 ns, total: 560 ms
Wall time: 558 ms


In [12]:
%%time
train_metadata['p0_region_4'] = train_metadata['p0_region_minus_4'] + train_metadata['p0_region_plus_4']
train_metadata['p0_region_3'] = train_metadata['p0_region_minus_3'] + train_metadata['p0_region_plus_3']
train_metadata['p0_region_2'] = train_metadata['p0_region_minus_2'] + train_metadata['p0_region_plus_2']
train_metadata['p0_region_1'] = train_metadata['p0_region_minus_1'] + train_metadata['p0_region_plus_1']

train_metadata['p1_region_4'] = train_metadata['p1_region_minus_4'] + train_metadata['p1_region_plus_4']
train_metadata['p1_region_3'] = train_metadata['p1_region_minus_3'] + train_metadata['p1_region_plus_3']
train_metadata['p1_region_2'] = train_metadata['p1_region_minus_2'] + train_metadata['p1_region_plus_2']
train_metadata['p1_region_1'] = train_metadata['p1_region_minus_1'] + train_metadata['p1_region_plus_1']

train_metadata['p2_region_4'] = train_metadata['p2_region_minus_4'] + train_metadata['p2_region_plus_4']
train_metadata['p2_region_3'] = train_metadata['p2_region_minus_3'] + train_metadata['p2_region_plus_3']
train_metadata['p2_region_2'] = train_metadata['p2_region_minus_2'] + train_metadata['p2_region_plus_2']
train_metadata['p2_region_1'] = train_metadata['p2_region_minus_1'] + train_metadata['p2_region_plus_1']

train_metadata['p3_region_4'] = train_metadata['p3_region_minus_4'] + train_metadata['p3_region_plus_4']
train_metadata['p3_region_3'] = train_metadata['p3_region_minus_3'] + train_metadata['p3_region_plus_3']
train_metadata['p3_region_2'] = train_metadata['p3_region_minus_2'] + train_metadata['p3_region_plus_2']
train_metadata['p3_region_1'] = train_metadata['p3_region_minus_1'] + train_metadata['p3_region_plus_1']

train_metadata['p4_region_4'] = train_metadata['p4_region_minus_4'] + train_metadata['p4_region_plus_4']
train_metadata['p4_region_3'] = train_metadata['p4_region_minus_3'] + train_metadata['p4_region_plus_3']
train_metadata['p4_region_2'] = train_metadata['p4_region_minus_2'] + train_metadata['p4_region_plus_2']
train_metadata['p4_region_1'] = train_metadata['p4_region_minus_1'] + train_metadata['p4_region_plus_1']

train_metadata['p5_region_4'] = train_metadata['p5_region_minus_4'] + train_metadata['p5_region_plus_4']
train_metadata['p5_region_3'] = train_metadata['p5_region_minus_3'] + train_metadata['p5_region_plus_3']
train_metadata['p5_region_2'] = train_metadata['p5_region_minus_2'] + train_metadata['p5_region_plus_2']
train_metadata['p5_region_1'] = train_metadata['p5_region_minus_1'] + train_metadata['p5_region_plus_1']



train_metadata['p0_region_4_percent'] = (train_metadata['p0_region_minus_4'] + train_metadata['p0_region_plus_4']) / train_metadata['number_of_0s']
train_metadata['p0_region_3_percent'] = (train_metadata['p0_region_minus_3'] + train_metadata['p0_region_plus_3']) / train_metadata['number_of_0s']
train_metadata['p0_region_2_percent'] = (train_metadata['p0_region_minus_2'] + train_metadata['p0_region_plus_2']) / train_metadata['number_of_0s'] 
train_metadata['p0_region_1_percent'] = (train_metadata['p0_region_minus_1'] + train_metadata['p0_region_plus_1']) / train_metadata['number_of_0s']

train_metadata['p1_region_4_percent'] = (train_metadata['p1_region_minus_4'] + train_metadata['p1_region_plus_4']) / train_metadata['number_of_1s']
train_metadata['p1_region_3_percent'] = (train_metadata['p1_region_minus_3'] + train_metadata['p1_region_plus_3']) / train_metadata['number_of_1s']
train_metadata['p1_region_2_percent'] = (train_metadata['p1_region_minus_2'] + train_metadata['p1_region_plus_2']) / train_metadata['number_of_1s']
train_metadata['p1_region_1_percent'] = (train_metadata['p1_region_minus_1'] + train_metadata['p1_region_plus_1']) / train_metadata['number_of_1s']

train_metadata['p2_region_4_percent'] = (train_metadata['p2_region_minus_4'] + train_metadata['p2_region_plus_4']) / train_metadata['number_of_2s']
train_metadata['p2_region_3_percent'] = (train_metadata['p2_region_minus_3'] + train_metadata['p2_region_plus_3']) / train_metadata['number_of_2s']
train_metadata['p2_region_2_percent'] = (train_metadata['p2_region_minus_2'] + train_metadata['p2_region_plus_2']) / train_metadata['number_of_2s']
train_metadata['p2_region_1_percent'] = (train_metadata['p2_region_minus_1'] + train_metadata['p2_region_plus_1']) / train_metadata['number_of_2s']

train_metadata['p3_region_4_percent'] = (train_metadata['p3_region_minus_4'] + train_metadata['p3_region_plus_4']) / train_metadata['number_of_3s']
train_metadata['p3_region_3_percent'] = (train_metadata['p3_region_minus_3'] + train_metadata['p3_region_plus_3']) / train_metadata['number_of_3s']
train_metadata['p3_region_2_percent'] = (train_metadata['p3_region_minus_2'] + train_metadata['p3_region_plus_2']) / train_metadata['number_of_3s']
train_metadata['p3_region_1_percent'] = (train_metadata['p3_region_minus_1'] + train_metadata['p3_region_plus_1']) / train_metadata['number_of_3s']

train_metadata['p4_region_4_percent'] = (train_metadata['p4_region_minus_4'] + train_metadata['p4_region_plus_4']) / train_metadata['number_of_4s']
train_metadata['p4_region_3_percent'] = (train_metadata['p4_region_minus_3'] + train_metadata['p4_region_plus_3']) / train_metadata['number_of_4s']
train_metadata['p4_region_2_percent'] = (train_metadata['p4_region_minus_2'] + train_metadata['p4_region_plus_2']) / train_metadata['number_of_4s']
train_metadata['p4_region_1_percent'] = (train_metadata['p4_region_minus_1'] + train_metadata['p4_region_plus_1']) / train_metadata['number_of_4s']

train_metadata['p5_region_4_percent'] = (train_metadata['p5_region_minus_4'] + train_metadata['p5_region_plus_4']) / train_metadata['number_of_5s']
train_metadata['p5_region_3_percent'] = (train_metadata['p5_region_minus_3'] + train_metadata['p5_region_plus_3']) / train_metadata['number_of_5s']
train_metadata['p5_region_2_percent'] = (train_metadata['p5_region_minus_2'] + train_metadata['p5_region_plus_2']) / train_metadata['number_of_5s']
train_metadata['p5_region_1_percent'] = (train_metadata['p5_region_minus_1'] + train_metadata['p5_region_plus_1']) / train_metadata['number_of_5s']





CPU times: user 33.5 ms, sys: 3.87 ms, total: 37.4 ms
Wall time: 35.2 ms


In [13]:
test_id = test_metadata_kaggle['object_id']

In [14]:
def multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def lgb_multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False

In [16]:
train_metadata[list(train_metadata.columns)[-48:]].head()

,p0_region_4,p0_region_3,p0_region_2,p0_region_1,p1_region_4,p1_region_3,p1_region_2,p1_region_1,p2_region_4,p2_region_3,p2_region_2,p2_region_1,p3_region_4,p3_region_3,p3_region_2,p3_region_1,p4_region_4,p4_region_3,p4_region_2,p4_region_1,p5_region_4,p5_region_3,p5_region_2,p5_region_1,p0_region_4_percent,p0_region_3_percent,p0_region_2_percent,p0_region_1_percent,p1_region_4_percent,p1_region_3_percent,p1_region_2_percent,p1_region_1_percent,p2_region_4_percent,p2_region_3_percent,p2_region_2_percent,p2_region_1_percent,p3_region_4_percent,p3_region_3_percent,p3_region_2_percent,p3_region_1_percent,p4_region_4_percent,p4_region_3_percent,p4_region_2_percent,p4_region_1_percent,p5_region_4_percent,p5_region_3_percent,p5_region_2_percent,p5_region_1_percent
0,0.0,0.0,33.0,30.0,0.0,0.0,23.0,35.0,0.0,0.0,27.0,31.0,0.0,0.0,28.0,30.0,0.0,0.0,28.0,30.0,0.0,0.0,32.0,25.0,0.000000,0.000000,0.523810,0.476190,0.000000,0.000000,0.396552,0.603448,0.000000,0.000000,0.465517,0.534483,0.000000,0.000000,0.482759,0.517241,0.000000,0.000000,0.482759,0.517241,0.000000,0.000000,0.561404,0.438596
1,0.0,1.0,24.0,45.0,0.0,0.0,23.0,33.0,0.0,0.0,21.0,35.0,0.0,0.0,22.0,34.0,0.0,0.0,20.0,36.0,0.0,1.0,21.0,34.0,0.000000,0.014286,0.342857,0.642857,0.000000,0.000000,0.410714,0.589286,0.000000,0.000000,0.375000,0.625000,0.000000,0.000000,0.392857,0.607143,0.000000,0.000000,0.357143,0.642857,0.000000,0.017857,0.375000,0.607143
2,1.0,0.0,22.0,49.0,1.0,0.0,13.0,38.0,3.0,1.0,2.0,46.0,3.0,0.0,2.0,47.0,3.0,1.0,1.0,46.0,2.0,2.0,5.0,42.0,0.013889,0.000000,0.305556,0.680556,0.019231,0.000000,0.250000,0.730769,0.057692,0.019231,0.038462,0.884615,0.057692,0.000000,0.038462,0.903846,0.058824,0.019608,0.019608,0.901961,0.039216,0.039216,0.098039,0.823529
3,2.0,3.0,8.0,59.0,1.0,0.0,1.0,54.0,1.0,1.0,0.0,54.0,1.0,2.0,1.0,52.0,1.0,3.0,1.0,51.0,1.0,3.0,1.0,50.0,0.027778,0.041667,0.111111,0.819444,0.017857,0.000000,0.017857,0.964286,0.017857,0.017857,0.000000,0.964286,0.017857,0.035714,0.017857,0.928571,0.017857,0.053571,0.017857,0.910714,0.018182,0.054545,0.018182,0.909091
4,1.0,1.0,16.0,45.0,2.0,1.0,3.0,52.0,2.0,1.0,3.0,52.0,2.0,1.0,2.0,53.0,2.0,1.0,0.0,55.0,2.0,1.0,0.0,54.0,0.015873,0.015873,0.253968,0.714286,0.034483,0.017241,0.051724,0.896552,0.034483,0.017241,0.051724,0.896552,0.034483,0.017241,0.034483,0.913793,0.034483,0.017241,0.000000,0.948276,0.035088,0.017544,0.000000,0.947368


In [17]:
#[0.5764, 0.54968, 0.59261, 0.56469, 0.5659]
"""
0 0.5742077279623574
1 0.5526378937737927
2 0.5895248906291187
3 0.5582664456933794
4 0.564887714048932
MULTI WEIGHTED LOG LOSS : 0.56791
"""

'\n0 0.5742077279623574\n1 0.5526378937737927\n2 0.5895248906291187\n3 0.5582664456933794\n4 0.564887714048932\nMULTI WEIGHTED LOG LOSS : 0.56791\n'

In [29]:
%%time
final_dict = {}

loss_list = []
temp = train_metadata_kaggle.copy()
temp = temp.merge(train_metadata[['object_id'] + used_columns1 ],on = 'object_id',how = 'left')

print(temp.shape)
y = temp['target']
del temp['target']
classes = sorted(y.unique())

class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'n_estimators': 4000, 
'subsample_freq': 6, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 40,
'min_child_weight': 150, 
'min_split_gain': 0.15, 
'num_leaves': 6, 
'reg_alpha': 0.1, 
'reg_lambda': 0.001, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

(7848, 143)
0 0.5782826478039932
1 0.5551127326134954
2 0.5903966629189505
3 0.5612577746680687
4 0.5680157481899087
MULTI WEIGHTED LOG LOSS : 0.57060 
CPU times: user 12min 26s, sys: 2.67 s, total: 12min 28s
Wall time: 1min 42s


In [21]:
list(imp_df.sort_values('gain',ascending=False)['feature'])

,feature,gain,fold
57,distmod,1084,5
55,hostgal_photoz,1073,5
54,mjd_diff_det,986,5
56,hostgal_photoz_err,623,5
61,hostgal_photoz_certain,599,5
46,flux__longest_strike_above_mean,438,5
52,mjd__mean_abs_change,420,5
75,__median_absolute_deviation___2_,377,5
3,flux_median,370,5
19,flux_diff2,360,5


In [ ]:
#final_dict2[final_dict2['column_name'].isin(most_imp_ones)]
#final_dict2[final_dict2['fold_sum'] == 1]

In [ ]:
#final_dict2.head()

In [ ]:
#final_dict2.to_csv('final_features3.csv',index=False)

In [ ]:
#modify to work with kfold
#def smoteAdataset(Xig, yig, test_size=0.2, random_state=0):
def smoteAdataset(Xig_train, yig_train, Xig_test, yig_test):
    
        
    sm=SMOTE(random_state=51)
    Xig_train_res, yig_train_res = sm.fit_sample(Xig_train, yig_train.ravel())

        
    return Xig_train_res, pd.Series(yig_train_res), Xig_test, pd.Series(yig_test)

In [ ]:
%%time
final_dict = {}

loss_list = []
temp = train_metadata_kaggle.copy()

#temp = temp.merge(train_metadata[['object_id'] + most_imp_ones2 ],on = 'object_id',how = 'left')

temp = temp.merge(train_metadata[['object_id'] + used_columns1  ],on = 'object_id',how = 'left')


print(temp.shape)
temp.fillna(0, inplace=True)

y = temp['target']
del temp['target']
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'n_estimators': 4000, 
'subsample_freq': 6, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 40,
'min_child_weight': 150, 
'min_split_gain': 0.15, 
'num_leaves': 6, 
'reg_alpha': 0.1, 
'reg_lambda': 0.001, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    trn_xa, trn_y, val_xa, val_y=smoteAdataset(trn_x.values, trn_y.values, val_x.values, val_y.values)
    trn_x=pd.DataFrame(data=trn_xa, columns=trn_x.columns)
    val_x=pd.DataFrame(data=val_xa, columns=val_x.columns)
    
    print(trn_x.shape,trn_y.shape,val_x.shape,val_y.shape)
    
    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

In [ ]:
#used_columns = used_columns1 + used_columns2 + used_columns3 

In [ ]:
print(train_metadata.shape,test_metadata.shape)

In [ ]:
#train_metadata.drop([x for x in train_metadata.columns if x not in ['object_id'] + used_columns ] ,axis = 1,inplace=True)

In [ ]:
#test_metadata = test_metadata[[x for x in test_metadata.columns if x in ['object_id'] + used_columns ]]

In [ ]:
train_metadata_kaggle = train_metadata_kaggle.merge(train_metadata[['object_id'] + used_columns1 ],on = 'object_id',how = 'left')
test_metadata_kaggle = test_metadata_kaggle.merge(test_metadata[['object_id'] + used_columns1 ],on = 'object_id',how = 'left')

In [ ]:
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

In [ ]:
test_metadata_kaggle.tail()

In [ ]:
test_metadata_kaggle.head()

In [ ]:
%%time
train_metadata_kaggle.to_csv('mydata_train_metadata.csv',index=False)
test_metadata_kaggle.to_csv('mydata_test_metadata.csv',index=False)

In [ ]:
imp_df.sort_values('gain',ascending=False).head()

In [ ]:
temp_test = test_metadata_kaggle.copy()

In [ ]:
del temp_test['object_id']

In [ ]:
temp_test.fillna(0,inplace = True)

In [ ]:
print(temp.shape,temp_test.shape)

In [ ]:
list(temp.columns) == list(temp_test.columns)

In [ ]:
%%time
test_pred0 = pd.DataFrame()
test_pred1 = pd.DataFrame()
test_pred2 = pd.DataFrame()
test_pred3 = pd.DataFrame()
test_pred4 = pd.DataFrame()

list_of_df = [test_pred0,test_pred1,test_pred2,test_pred3,test_pred4]

for num,c in enumerate(clfs):
    print(num)
    for k in range(0,len(temp_test),500000):
        test_pred = pd.DataFrame(c.predict_proba(temp_test[ k:k+500000] ))
        list_of_df[num] = pd.concat([list_of_df[num],test_pred],axis=0)
        del test_pred

In [ ]:
test_pred2 = pd.DataFrame()
test_pred2 = (list_of_df[0] + list_of_df[1] + list_of_df[2] + list_of_df[3] + list_of_df[4])/5

In [ ]:
print(test_pred2.shape)

In [ ]:
#test_pred2 = pd.DataFrame(np.random.rand(10,14))

In [ ]:
test_pred2.head()

In [ ]:
temp_columns = ['object_id','class_6','class_15','class_16','class_42','class_52','class_53','class_62','class_64','class_65','class_67','class_88','class_90','class_92','class_95','class_99']

In [ ]:
test_pred2.columns = temp_columns[1:15]

In [ ]:
def getUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

In [ ]:
klm = pd.DataFrame()
klm['mymean'] = test_pred2[feats].mean(axis=1)
klm['mymedian'] = test_pred2[feats].median(axis=1)
klm['mymax'] = test_pred2[feats].max(axis=1)

In [ ]:
test_pred2['class_99'] = getUnknown(klm)

In [ ]:
test_pred2.tail()

In [ ]:
test_pred2 = test_pred2.reset_index(drop=True)

In [ ]:
print(test_pred2.shape,test_id.shape)

In [ ]:
test_id.tail()

In [ ]:
test_id = test_id.reset_index(drop=True)

In [ ]:
test_id.index == test_pred2.index

In [ ]:
%%time
test_pred = pd.concat([test_id,test_pred2],axis=1)

In [ ]:
test_pred = test_pred[temp_columns]

In [ ]:
test_pred.head()

In [ ]:
print(test_pred.shape)

In [ ]:
%%time
test_pred.to_csv('test_pred_40.csv',index=False)

In [ ]:
#!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_27.csv -m "Message"